# 🔧 Training Improvements Applied

## Changes Made to Improve Learning:

### 1. **Learning Rate Reduced**: `1e-3` → `1e-4`

- Lower learning rate provides more stable gradient updates
- Helps prevent overshooting optimal weights
- Better for partial label scenarios

### 2. **Class Weights Added to Loss Function**

- Calculated from validation set class distribution
- Handles class imbalance (e.g., background vs buildings)
- Prevents model from collapsing to majority class
- Uses inverse frequency weighting

### Expected Improvements:

- ✅ Validation mIoU should **increase** across epochs (not decrease)
- ✅ Per-class IoU should be more balanced
- ✅ Model should learn features for minority classes
- ✅ More stable training dynamics

**Note**: These changes should make the model visibly learn better, even with only 4 epochs.


# Notebook 3: Training Experiments

Full experimental suite comparing different label fractions with partial supervision.

## Experiments:

- **Label fractions**: 30%, 50%, 70%
- **Training mode**: Partial Cross Entropy Loss (supervised only on labeled pixels)
- **Architecture**: UNet with 5 output classes
- **Total runs**: 3 experiments (30 epochs each)


In [1]:
import sys
from pathlib import Path
sys.path.append('..')

In [2]:
try:
    from src import (
        set_seed, mask_to_rgb, visualize_sample, plot_training_history, save_checkpoint,
        LandCoverDataset, mask_labels_random, get_train_transform, get_val_transform,
        get_unet,
        PartialCrossEntropyLoss,
        compute_iou, compute_pixel_accuracy,
        Trainer
    )
    from tqdm.auto import tqdm
    from torch.utils.data import DataLoader
    import json
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np
    import torch.nn as nn
    import torch
except ImportError as e:
    print(
        f"ImportError: {e}. Please ensure all required modules are available.")


set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Random seed set to 42
Using device: cuda


In [ ]:
# Enable cuDNN auto-tuner for faster training
# This finds the best convolution algorithms for your specific GPU and input sizes
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

print("⚡ Performance Optimizations Enabled:")
print(f"  cuDNN benchmark: {torch.backends.cudnn.benchmark}")
print(f"  cuDNN enabled: {torch.backends.cudnn.enabled}")
print("\nThis will:")
print("  - Auto-tune convolution algorithms for our GPU")
print("  - Provide 10-20% speedup on first epoch")
print("  - Much faster on subsequent epochs")

In [ ]:
# Verify GPU is being used and check memory
if torch.cuda.is_available():
    print(f"\n✅ GPU Information:")
    print(f"  Device: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")
    print(
        f"  Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(
        f"  Current Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(
        f"  Current Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

    # Check if we should use AMP (Automatic Mixed Precision)
    cuda_capability = torch.cuda.get_device_capability(0)
    # Tensor cores available on compute capability >= 7.0
    amp_supported = cuda_capability[0] >= 7

    if amp_supported:
        print(f"\n⚡ Your GPU supports Automatic Mixed Precision (AMP)")
        print(
            f"  Compute Capability: {cuda_capability[0]}.{cuda_capability[1]}")
        print(f"  Expected speedup: 1.5-2.5x with negligible accuracy loss")
        print(f"  Memory savings: ~30-40%")
    else:
        print(f"\n⚠️  Your GPU has limited AMP support")
        print(
            f"  Compute Capability: {cuda_capability[0]}.{cuda_capability[1]}")
else:
    print("\n❌ WARNING: No GPU detected! Training will be VERY slow.")
    print("   Please ensure CUDA is installed and GPU drivers are up to date.")

## Configuration


In [ ]:
DEBUG = False

DATA_ROOT = Path("../data")

CONFIG = {
    'num_classes': 5,
    'ignore_index': -1,
    'batch_size': 4,
    'num_workers': 4,
    'epochs': 4 if not DEBUG else 1,
    'learning_rate': 1e-4,  # Reduced from 1e-3 to improve stability
    'weight_decay': 1e-4,
    'architecture': 'unetplusplus',
    'seed': 42,
    'use_amp': True  # Enable Automatic Mixed Precision for 2x speedup
}

LABEL_FRACTIONS = [0.3, 0.5, 0.7]

RESULTS_DIR = Path("../runs")
RESULTS_DIR.mkdir(exist_ok=True)

print("Configuration:")
print(json.dumps(CONFIG, indent=2))
print(f"\nLabel fractions: {LABEL_FRACTIONS}")
print(f"Total experiments: {len(LABEL_FRACTIONS)}")
print(f"DEBUG mode: {DEBUG}")

Configuration:
{
  "num_classes": 5,
  "ignore_index": -1,
  "batch_size": 4,
  "num_workers": 4,
  "epochs": 4,
  "learning_rate": 0.001,
  "weight_decay": 0.0001,
  "architecture": "unet",
  "seed": 42
}

Label fractions: [0.3, 0.5, 0.7]
Total experiments: 3
DEBUG mode: False


## Dataset Info

The dataset uses pre-defined train/val/test splits:

- Train: 7,471 patches
- Val: 1,603 patches
- Test: 1,603 patches

All patches are 512x512 pixels extracted from 41 large tiles.


In [4]:
# Verify dataset availability
print(f"Data directory: {DATA_ROOT}")
print(f"Images: {(DATA_ROOT / 'images').exists()}")
print(f"Masks: {(DATA_ROOT / 'masks').exists()}")
print(f"Train split: {(DATA_ROOT / 'train.txt').exists()}")
print(f"Val split: {(DATA_ROOT / 'val.txt').exists()}")
print(f"Test split: {(DATA_ROOT / 'test.txt').exists()}")

# Count patches in each split
if (DATA_ROOT / 'train.txt').exists():
    with open(DATA_ROOT / 'train.txt') as f:
        train_count = len(f.readlines())
    with open(DATA_ROOT / 'val.txt') as f:
        val_count = len(f.readlines())
    with open(DATA_ROOT / 'test.txt') as f:
        test_count = len(f.readlines())

    print(f"\n✓ Train patches: {train_count}")
    print(f"✓ Val patches: {val_count}")
    print(f"✓ Test patches: {test_count}")
else:
    print("\n⚠ Split files not found!")

Data directory: ..\data
Images: True
Masks: True
Train split: True
Val split: True
Test split: True

✓ Train patches: 7470
✓ Val patches: 1602
✓ Test patches: 1602


## Experiment Loop


In [ ]:
# Calculate class weights from validation set (has 100% labels)
# This helps handle class imbalance issues
import numpy as np
from collections import Counter
print("Calculating class weights from validation set...")


# Load a subset of validation data to compute class distribution
temp_val_dataset = LandCoverDataset(
    data_dir=DATA_ROOT,
    split='val',
    transform=None,  # No transform for accurate counting
    labeled_fraction=1.0,
    seed=CONFIG['seed'],
    use_split_file=True
)

# Count pixels per class (sample 100 images to save time)
class_counts = Counter()
sample_size = min(100, len(temp_val_dataset))

for idx in range(sample_size):
    _, mask = temp_val_dataset[idx]
    unique, counts = np.unique(mask.numpy(), return_counts=True)
    for cls, count in zip(unique, counts):
        if cls != -1:  # Skip ignore index
            class_counts[cls] += count

# Calculate inverse frequency weights
total_pixels = sum(class_counts.values())
class_weights = []

print(f"\nClass distribution (from {sample_size} validation images):")
print("-" * 60)

class_names = ["Building", "Woodland", "Water", "Road", "Background"]

for cls in range(CONFIG['num_classes']):
    count = class_counts.get(cls, 1)  # Avoid division by zero
    percentage = (count / total_pixels) * 100
    weight = total_pixels / \
        (CONFIG['num_classes'] * count)  # Inverse frequency
    class_weights.append(weight)
    print(
        f"{class_names[cls]:12s} (class {cls}): {count:10,} pixels ({percentage:5.2f}%) → weight: {weight:.4f}")

# Convert to tensor
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Normalize weights so mean = 1.0 (optional, helps with learning rate)
class_weights = class_weights / class_weights.mean()

print(f"\nNormalized class weights: {class_weights.tolist()}")
print(f"These weights will be applied to the loss function to handle class imbalance.")
print("-" * 60)

In [ ]:
all_results = []

for label_frac in LABEL_FRACTIONS:
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: label_frac={label_frac}")
    print(f"{'='*70}\n")

    train_dataset = LandCoverDataset(
        data_dir=DATA_ROOT,
        split='train',
        transform=get_train_transform(),
        labeled_fraction=label_frac,
        seed=CONFIG['seed'],
        use_split_file=True
    )

    val_dataset = LandCoverDataset(
        data_dir=DATA_ROOT,
        split='val',
        transform=get_val_transform(),
        labeled_fraction=1.0,
        seed=CONFIG['seed'],
        use_split_file=True
    )

    print(f"Train dataset: {len(train_dataset)} patches")
    print(f"Val dataset: {len(val_dataset)} patches")

    # Adjust num_workers for DEBUG mode
    num_workers = 0 if DEBUG else CONFIG['num_workers']

    if DEBUG:
        from torch.utils.data import Subset
        train_dataset = Subset(train_dataset, range(8))
        val_dataset = Subset(val_dataset, range(4))
        print(
            f"\nDEBUG: Using {len(train_dataset)} train, {len(val_dataset)} val samples")
        print(f"DEBUG: num_workers set to {num_workers} (required for Subset)")

    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    model = get_unet(
        model_type=CONFIG['architecture'],
        classes=CONFIG['num_classes'],
        in_channels=3
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=CONFIG['learning_rate'],
        weight_decay=CONFIG['weight_decay']
    )

    # Use class weights to handle imbalance
    criterion = PartialCrossEntropyLoss(
        ignore_index=CONFIG['ignore_index'],
        weight=class_weights.to(device)
    )

    exp_dir = RESULTS_DIR / f"frac{int(label_frac*100)}_partial_ce"
    exp_dir.mkdir(exist_ok=True, parents=True)

    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        mode='partial_ce',
        num_classes=CONFIG['num_classes'],
        ignore_index=CONFIG['ignore_index'],
        save_dir=str(exp_dir),
        use_amp=CONFIG.get('use_amp', False)  # Enable AMP for speedup
    )

    print(f"\nStarting training for label_fraction={label_frac}")
    history = trainer.fit(num_epochs=CONFIG['epochs'], save_best=True)

    all_results.append({
        'label_fraction': label_frac,
        'history': history,
        'final_val_miou': history['val_miou'][-1],
        'best_val_miou': max(history['val_miou']),
        'final_val_acc': history['val_acc'][-1],
        'exp_dir': str(exp_dir)
    })

    plot_training_history(history, save_path=exp_dir / 'training_history.png')
    print(f"\n✓ Experiment complete: {exp_dir}")

print("\n" + "="*70)
print("ALL EXPERIMENTS COMPLETED")
print("="*70)


EXPERIMENT: label_frac=0.3

✓ Loaded 7470 patches from train.txt
✓ Loaded 1602 patches from val.txt
Train dataset: 7470 patches
Val dataset: 1602 patches
Created local UNet implementation

Starting training for label_fraction=0.3

Starting training for 4 epochs
Mode: partial_ce
Device: cuda
Train batches: 1868
Val batches: 401
------------------------------------------------------------

Starting training for label_fraction=0.3

Starting training for 4 epochs
Mode: partial_ce
Device: cuda
Train batches: 1868
Val batches: 401
------------------------------------------------------------


Epoch 1 [Train]:   4%|▍         | 73/1868 [03:16<1:20:30,  2.69s/it, loss=1.1615]



KeyboardInterrupt: 

## Results Summary Table


In [ ]:
# Create summary DataFrame
summary_data = []
for r in all_results:
    summary_data.append({
        'Label Fraction': f"{int(r['label_fraction']*100)}%",
        'Final mIoU': f"{r['final_val_miou']:.4f}",
        'Best mIoU': f"{r['best_val_miou']:.4f}",
        'Final Accuracy': f"{r['final_val_acc']:.4f}"
    })

df_summary = pd.DataFrame(summary_data)
print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)
print(df_summary.to_string(index=False))
print("="*60)

## Training Curves Visualization


In [ ]:
# Extract data for plotting
label_fracs = [r['label_fraction'] for r in all_results]
final_mious = [r['final_val_miou'] for r in all_results]
best_mious = [r['best_val_miou'] for r in all_results]

# Create plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: mIoU vs Label Fraction
axes[0].plot(label_fracs, final_mious, 'o-',
             label='Final mIoU', linewidth=2, markersize=8)
axes[0].plot(label_fracs, best_mious, 's--',
             label='Best mIoU', linewidth=2, markersize=8)
axes[0].set_xlabel('Label Fraction', fontsize=12)
axes[0].set_ylabel('mIoU', fontsize=12)
axes[0].set_title('Performance vs Label Fraction',
                  fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(label_fracs)
axes[0].set_xticklabels([f'{int(f*100)}%' for f in label_fracs])

# Plot 2: Training curves for best experiment
best_exp = max(all_results, key=lambda x: x['best_val_miou'])
history = best_exp['history']
epochs = range(1, len(history['train_loss']) + 1)

axes[1].plot(epochs, history['train_loss'], label='Train Loss', linewidth=2)
axes[1].plot(epochs, history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title(f"Best Model Training Curve ({int(best_exp['label_fraction']*100)}% labels)",
                  fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'comparison_plots.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Comparison plots saved to: {RESULTS_DIR / 'comparison_plots.png'}")

## Training History: All Label Fractions

Comparison of training dynamics across different label fractions (30%, 50%, 70%).


In [ ]:
# Create detailed comparison plots for all experiments
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Training History Comparison Across All Experiments',
             fontsize=16, fontweight='bold')

# Define colors for each label fraction
colors = {0.3: 'blue', 0.5: 'green', 0.7: 'red'}

# Plot 1: Training Loss
for r in all_results:
    label_frac = r['label_fraction']
    history = r['history']
    epochs = range(1, len(history['train_loss']) + 1)
    axes[0, 0].plot(epochs, history['train_loss'],
                    label=f"{int(label_frac*100)}%",
                    color=colors[label_frac], linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=11)
axes[0, 0].set_ylabel('Loss', fontsize=11)
axes[0, 0].set_title('Training Loss', fontsize=13, fontweight='bold')
axes[0, 0].legend(title='Label Fraction', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Loss
for r in all_results:
    label_frac = r['label_fraction']
    history = r['history']
    epochs = range(1, len(history['val_loss']) + 1)
    axes[0, 1].plot(epochs, history['val_loss'],
                    label=f"{int(label_frac*100)}%",
                    color=colors[label_frac], linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=11)
axes[0, 1].set_ylabel('Loss', fontsize=11)
axes[0, 1].set_title('Validation Loss', fontsize=13, fontweight='bold')
axes[0, 1].legend(title='Label Fraction', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Validation mIoU
for r in all_results:
    label_frac = r['label_fraction']
    history = r['history']
    epochs = range(1, len(history['val_miou']) + 1)
    axes[1, 0].plot(epochs, history['val_miou'],
                    label=f"{int(label_frac*100)}%",
                    color=colors[label_frac], linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=11)
axes[1, 0].set_ylabel('mIoU', fontsize=11)
axes[1, 0].set_title('Validation mIoU', fontsize=13, fontweight='bold')
axes[1, 0].legend(title='Label Fraction', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Validation Accuracy
for r in all_results:
    label_frac = r['label_fraction']
    history = r['history']
    epochs = range(1, len(history['val_acc']) + 1)
    axes[1, 1].plot(epochs, history['val_acc'],
                    label=f"{int(label_frac*100)}%",
                    color=colors[label_frac], linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontsize=11)
axes[1, 1].set_ylabel('Accuracy', fontsize=11)
axes[1, 1].set_title('Validation Pixel Accuracy',
                     fontsize=13, fontweight='bold')
axes[1, 1].legend(title='Label Fraction', fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'training_curves_all.png',
            dpi=150, bbox_inches='tight')
plt.show()

print(f"Training curves saved to: {RESULTS_DIR / 'training_curves_all.png'}")

## Qualitative Results: Predictions Visualization


In [ ]:
# Use validation dataset for visualization
vis_dataset = LandCoverDataset(
    data_dir=DATA_ROOT,
    split='val',
    transform=get_val_transform(),
    labeled_fraction=1.0,
    use_split_file=True
)

print(f"Visualization dataset: {len(vis_dataset)} patches")

# Get first 3 validation samples
vis_indices = list(range(min(3, len(vis_dataset))))

# Select best experiment to visualize (70% labels)
best_exp = [r for r in all_results if r['label_fraction'] == 0.7][0]
best_model_path = Path(best_exp['exp_dir']) / 'best_model.pth'

# Load model
vis_model = get_unet(
    model_type=CONFIG['architecture'],
    classes=CONFIG['num_classes'],
    in_channels=3
)
checkpoint = torch.load(
    best_model_path, map_location=device, weights_only=False)
vis_model.load_state_dict(checkpoint['model_state_dict'])
vis_model = vis_model.to(device)
vis_model.eval()

# Create visualization
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Qualitative Results: Best Model (70% labels)',
             fontsize=16, fontweight='bold')

for i in vis_indices:
    image, mask_gt = vis_dataset[i]
    image_input = image.unsqueeze(0).to(device)

    # Get prediction
    with torch.no_grad():
        output = vis_model(image_input)
        pred = output.argmax(dim=1).squeeze(0).cpu()

    # Convert to numpy for visualization
    img_np = image.cpu().permute(1, 2, 0).numpy()
    # Denormalize
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_np = std * img_np + mean
    img_np = np.clip(img_np, 0, 1)

    mask_gt_rgb = mask_to_rgb(mask_gt.numpy())
    pred_rgb = mask_to_rgb(pred.numpy())

    # Create partially masked GT for comparison (30% labels)
    masked_gt = mask_labels_random(mask_gt.numpy(), 0.3, seed=42)
    masked_gt_rgb = mask_to_rgb(masked_gt)

    # Plot
    axes[i, 0].imshow(img_np)
    axes[i, 0].set_title('Input Image', fontsize=11)
    axes[i, 0].axis('off')

    axes[i, 1].imshow(mask_gt_rgb)
    axes[i, 1].set_title('Ground Truth', fontsize=11)
    axes[i, 1].axis('off')

    axes[i, 2].imshow(masked_gt_rgb)
    axes[i, 2].set_title('Partial Labels (30%)', fontsize=11)
    axes[i, 2].axis('off')

    axes[i, 3].imshow(pred_rgb)
    axes[i, 3].set_title('Prediction', fontsize=11)
    axes[i, 3].axis('off')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'qualitative_results.png',
            dpi=150, bbox_inches='tight')
plt.show()

print(
    f"Qualitative results saved to: {RESULTS_DIR / 'qualitative_results.png'}")

## Per-Class IoU Analysis


In [ ]:
class_names = ['Background', 'Buildings', 'Woodlands', 'Water', 'Roads']

per_class_data = []
for r in all_results:
    history = r['history']
    best_epoch_idx = np.argmax(history['val_miou'])

    row = {'Experiment': f"{int(r['label_fraction']*100)}% Labels"}

    for i, class_name in enumerate(class_names):
        class_key = f'val_class_{i}_iou'
        if class_key in history:
            row[class_name] = history[class_key][best_epoch_idx]
        else:
            row[class_name] = 0.0

    per_class_data.append(row)

df_per_class = pd.DataFrame(per_class_data)

fig, ax = plt.subplots(figsize=(12, 6))

heatmap_data = df_per_class[class_names].values
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt='.3f',
    cmap='RdYlGn',
    vmin=0.0,
    vmax=1.0,
    xticklabels=class_names,
    yticklabels=df_per_class['Experiment'].values,
    cbar_kws={'label': 'IoU Score'},
    ax=ax
)

ax.set_title('Per-Class IoU Comparison at Best Epoch',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Class', fontsize=12)
ax.set_ylabel('Experiment', fontsize=12)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'per_class_iou.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Per-class IoU heatmap saved to: {RESULTS_DIR / 'per_class_iou.png'}")
print("\nPer-Class IoU Table:")
print(df_per_class.to_string(index=False))

---

## 📊 Experiment Summary

**All experiments completed successfully!**

### Approach:

This notebook implements **partial supervision** for semantic segmentation:

- **Partial labels**: Only a fraction of pixels (30%, 50%, 70%) have ground truth labels
- **Unlabeled pixels**: Masked with `ignore_index=-1` and excluded from loss computation
- **Loss function**: PartialCrossEntropyLoss (standard cross-entropy on labeled pixels only)
- **Training mode**: Fully supervised on the available labeled pixels (not semi-supervised)

### Key Questions:

1. **Effect of Label Fraction**:

   - How does increasing labeled pixels from 30% → 50% → 70% affect performance?
   - Is the improvement linear or does it saturate?

2. **Model Performance**:

   - What mIoU can we achieve with only 30% labeled pixels?
   - How much does performance improve with more labels?

3. **Per-Class Analysis**:
   - Which classes are most affected by limited labels?
   - Are some classes easier to segment with partial supervision?

### Next Steps:

- Analyze per-class performance differences
- Compare with fully supervised baseline (100% labels)
- Explore intelligent label selection strategies (e.g., uncertainty sampling, active learning)
